In [1]:
from weavepy import *
from config import BDD_VERSION
sns.set_context("talk")

ERROR 1: PROJ: proj_create_from_database: Open of /net/nfs/tools/u20/Python/miniconda3_py311_23.11.0-2/envs/pangeo-meso-2024.01.22/share/proj failed


In [2]:
# --- Parameters for loading data ---
# It is expected that these will be changed rarely
## Variables
climate_variable = "TA"
energy_variable = "SPV"
techno = "60" # Techno needs to be specified for energy variables
## Temporal aggregation
aggregation_frequency = "D"
aggregation_function = "mean"
## Selection
country = "FR"
models = []
scenarios = []

In [3]:
data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)

Loading country n°1/1 (FR)
|_Variable n°1/2 (TA)
|__Techno n°1/1 (NA)
|__ > File already exists, loading from cache: /data/sbourdin/PECD42_cache_data/PECD42/TA_NA_D-mean_FR_PECD42.nc
|_Variable n°2/2 (SPV)
|__Techno n°1/1 (60)
|__ > File already exists, loading from cache: /data/sbourdin/PECD42_cache_data/PECD42/SPV_60_D-mean_FR_PECD42.nc


In [4]:
# --- Period ---
# These can be changed and adjusted easily
historical_period = (1990, 2020)
future_period = (2070, 2100)

In [5]:
# --- Thresholds ---
# These can be changed and adjusted easily
## Climate
climate_variable_comparison = '<=' # '<', '<=', '>', or '>='
climate_variable_threshold = 5 # Number in units of variable
## Energy 
energy_variable_comparison = '<' # '<', '<=', '>', or '>='
energy_variable_threshold = 0.05 # Number in units of variable

In [6]:
# Identify problematic days
climate_days = identify_pb_days(data[climate_variable], climate_variable_comparison, climate_variable_threshold).rename("climate_event")
energy_days = identify_pb_days(data[energy_variable], energy_variable_comparison, energy_variable_threshold).rename("energy_event")
compound_days = (climate_days & energy_days).rename("compound_event")

In [7]:
# Identify events
climate_events = identify_events_whole_base(climate_days)
energy_events = identify_events_whole_base(energy_days)
compound_events = identify_events_whole_base(compound_days)

  3%|▎         | 1/35 [00:00<00:00, 36.80it/s]


AttributeError: module 'numpy' has no attribute 'concat'

In [ ]:
# Compute number of events per year
N_climate_events, N_energy_events, N_compound_events = [count_events(e) for e in [climate_events, energy_events, compound_events]]

In [ ]:
nb_event_timeseries_plotly_multi([N_climate_events, N_energy_events, N_compound_events])

In [ ]:
event_duration_hist(climate_events, energy_events, compound_events)

In [ ]:
event_seasonality_kde(climate_days, energy_days, compound_days)